# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Read the CSV file
citiData = pd.read_csv("../output/cities.csv")
citiData.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Busselton,-33.65,115.33,55.99,84,10,13.87,AU,1594582566
1,1,Aleksandrov Gay,50.15,48.57,75.72,53,81,14.72,RU,1594582634
2,2,Bluff,-46.60,168.33,39.00,83,85,3.00,NZ,1594582634
3,3,Kaitangata,-46.28,169.85,37.00,75,7,5.01,NZ,1594582634
4,4,Saint Paul Harbor,57.79,-152.41,55.40,82,90,6.93,US,1594582634


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' into  locations 
locations = citiData[["Lat", "Lng"]]
humidity = citiData["Humidity"]

In [4]:
# Create a Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=85,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Narrow down the cities data which fits our choice of weather conditions
weatherData = citiData.loc[(citiData["Max Temp"] > 70) & (citiData["Max Temp"] < 80) & (citiData["Wind Speed"] < 10) & (citiData["Cloudiness"] == 0)]

# Drop the rows with null value, if any
weatherData = weatherData.dropna()

weatherData


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
38,38,Ponta do Sol,32.67,-17.10,73.00,73,0,4.70,PT,1594582579
187,187,Nouadhibou,20.93,-17.03,77.00,69,0,8.05,MR,1594582646
219,219,Saint Lawrence,36.06,14.20,77.00,78,0,6.93,MT,1594582649
333,333,Ilhabela,-23.78,-45.36,77.41,63,0,3.06,BR,1594582617
365,365,Carthage,36.86,10.33,78.80,54,0,4.70,TN,1594582659
477,477,Masallātah,32.62,14.00,71.73,82,0,4.09,LY,1594582666
487,487,Orán,-23.13,-64.33,75.99,46,0,5.10,AR,1594582666
563,563,Buri,-23.80,-48.59,72.73,66,0,7.54,BR,1594582671


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
# Set the weather data into a new variable
hotel_df = weatherData

# Add a new column in the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
38,38,Ponta do Sol,32.67,-17.10,73.00,73,0,4.70,PT,1594582579,
187,187,Nouadhibou,20.93,-17.03,77.00,69,0,8.05,MR,1594582646,
219,219,Saint Lawrence,36.06,14.20,77.00,78,0,6.93,MT,1594582649,
333,333,Ilhabela,-23.78,-45.36,77.41,63,0,3.06,BR,1594582617,
365,365,Carthage,36.86,10.33,78.80,54,0,4.70,TN,1594582659,
477,477,Masallātah,32.62,14.00,71.73,82,0,4.09,LY,1594582666,
487,487,Orán,-23.13,-64.33,75.99,46,0,5.10,AR,1594582666,
563,563,Buri,-23.80,-48.59,72.73,66,0,7.54,BR,1594582671,


In [7]:

# geocoordinates
target_radius = 5000
target_type = "lodging"

# set up a parameters dictionary
params = {   
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    }

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# For each row, column in the DataFrame...
for index, row in hotel_df.iterrows():
    
    citi_lat = row["Lat"]
    citi_lng = row["Lng"]
    
    params["location"] = f"{citi_lat},{citi_lng}"
    
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params = params).json()
    
    try:
        
        print(f"Closest hotel is {response['results'][0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    
    except:
        
        print("Missing field/result... skipping.")


Retrieving Results for Index 38: Ponta do Sol.
Closest hotel is Hotel do Campo.
Retrieving Results for Index 187: Nouadhibou.
Closest hotel is El Medina.
Retrieving Results for Index 219: Saint Lawrence.
Closest hotel is San Andrea Hotel.
Retrieving Results for Index 333: Ilhabela.
Closest hotel is Ilha Flat Hotel.
Retrieving Results for Index 365: Carthage.
Closest hotel is Dar Fatma.
Retrieving Results for Index 477: Masallātah.
Closest hotel is Mohamed Ahmed Salem.
Retrieving Results for Index 487: Orán.
Closest hotel is Hotel Residencial Centro.
Retrieving Results for Index 563: Buri.
Closest hotel is Hotel Aro.


In [8]:
# Display the DataFrame
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
38,38,Ponta do Sol,32.67,-17.10,73.00,73,0,4.70,PT,1594582579,Hotel do Campo
187,187,Nouadhibou,20.93,-17.03,77.00,69,0,8.05,MR,1594582646,El Medina
219,219,Saint Lawrence,36.06,14.20,77.00,78,0,6.93,MT,1594582649,San Andrea Hotel
333,333,Ilhabela,-23.78,-45.36,77.41,63,0,3.06,BR,1594582617,Ilha Flat Hotel
365,365,Carthage,36.86,10.33,78.80,54,0,4.70,TN,1594582659,Dar Fatma
477,477,Masallātah,32.62,14.00,71.73,82,0,4.09,LY,1594582666,Mohamed Ahmed Salem
487,487,Orán,-23.13,-64.33,75.99,46,0,5.10,AR,1594582666,Hotel Residencial Centro
563,563,Buri,-23.80,-48.59,72.73,66,0,7.54,BR,1594582671,Hotel Aro


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""

# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:

# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))